In [ ]:
!pip install pymongo pandas pillow tqdm scrapy selenium selenium-wire beautifulsoup4 ipywidgets requests twitter-scraper-selenium ipython
!pip install scrapy #
!pip install -U selenium
!pip install webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6

  Using cached selenium-4.28.1-py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.7.0
    Uninstalling selenium-4.7.0:
      Successfully uninstalled selenium-4.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
twitter-scraper-selenium 6.2.2 requires selenium==4.7.0, but you have selenium 4.28.1 which is incompatible.
ERROR: Operation cancelled by user


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cookie_ins.txt to cookie_ins.txt


In [ ]:
# 导入包(PypI)
import os
import re
import time
import json
import random
import pymongo
import pandas as pd

from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm.notebook import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import requests

/usr/local/lib/python3.11/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
import time

def load_cookies(browser, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:  # 指定编码，避免 UnicodeDecodeError
        for line in file:
            if not line.startswith('#') and line.strip():  # 跳过注释和空行
                fields = line.strip().split('\t')
                if len(fields) == 7:
                    cookie = {
                        'domain': fields[0].lstrip('.'),  # 去掉域名前的点
                        'path': fields[2],
                        'secure': fields[3].lower() == 'true',  # 转换为布尔值
                        'expiration': fields[4],
                        'name': fields[5],
                        'value': fields[6]
                    }

                    # 处理 `expiration` 可能为空的问题
                    expiry = int(cookie['expiration']) if cookie['expiration'].isdigit() else None

                    # 添加 Cookie
                    try:
                        browser.add_cookie({
                            'name': cookie['name'],
                            'value': cookie['value'],
                            'domain': cookie['domain'],
                            'path': cookie['path'],
                            'secure': cookie['secure'],
                            'expiry': expiry
                        })
                    except Exception as e:
                        print(f"⚠️ 无法添加 Cookie: {cookie}, 错误: {e}")

    print("✅ 所有 Cookies 已成功加载！")

In [ ]:
def scroll_until_posts_loaded(browser, target_post_count, max_scroll_attempts=50):
    """
    滚动页面，收集目标数量的帖子，并打印收集的帖子链接（终极版本）。
    """

    collected_posts = []  # **✅ 用列表存储帖子，确保顺序**
    seen_links = set()  # **✅ 用集合去重**
    scroll_attempts = 0

    time.sleep(5)  # **等待页面初始化加载帖子**

    while len(collected_posts) < target_post_count and scroll_attempts < max_scroll_attempts:
        # ✅ **每次滚动后获取当前所有可见帖子**
        posts = browser.find_elements(By.XPATH, '//a[contains(@href, "/p/") or contains(@href, "/reel/")]')

        new_posts_found = 0  # **统计本次新增的帖子**

        for post in posts:
            try:
                link = post.get_attribute('href')
                if link and ("/p/" in link or "/reel/" in link):
                    if link not in seen_links:
                        seen_links.add(link)  # **添加到集合，去重**
                        collected_posts.append(link)  # **✅ 保持原始顺序**
                        new_posts_found += 1
                        print(f"🆕 新帖子收集: {link}")  # ✅ **打印新收集的帖子**
            except Exception as e:
                print(f"⚠️ Error extracting link: {e}")

        print(f"✅ 本次滚动新增 {new_posts_found} 个帖子，总计: {len(collected_posts)}")  # ✅ **打印每次滚动后收集的帖子数**
        print(f"📋 当前收集的所有帖子: {collected_posts}")  # ✅ **打印完整列表，看看是不是有遗漏**

        # ✅ **如果帖子足够，就停止**
        if len(collected_posts) >= target_post_count:
            print("🎯 目标帖子数量已收集，停止滚动！")
            break

        # ✅ **进行小幅滚动，让 Instagram 逐步加载**
        browser.execute_script("window.scrollBy(0, 800);")
        time.sleep(random.uniform(3, 6))  # **等待加载**

        scroll_attempts += 1

    print(f"🎯 成功收集 {len(collected_posts)} 篇帖子")  # ✅ **最终结果**
    print(f"🔍 最终帖子列表: {collected_posts}")  # **最后打印所有收集到的帖子**

    return collected_posts[:target_post_count]  # **返回按顺序存储的帖子**



def download_image(url, save_path):
        """
        下载图片并保存到指定路径。

        Args:
            url (str): 图片的URL地址。
            save_path (str): 图片保存的本地路径。

        Returns:
            bool: 下载是否成功。
        """
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # 检查请求是否成功
            with open(save_path, 'wb') as f:
                f.write(response.content)
            return True
        except Exception as e:
            print(f"下载图片时出错，URL: {url}, 错误: {str(e)}")
            return False

def scrape_instagram(link="https://www.instagram.com/aloyoga/", num_posts_to_scrape=100):
    browser.get(link)

    # ✅ 获取帖子链接
    post_links = scroll_until_posts_loaded(browser, num_posts_to_scrape, 50)
    print(f"🚀 Successfully collected {len(post_links)} posts")
    print(post_links)

    if len(post_links) < num_posts_to_scrape:
        print(f"⚠️ Only {len(post_links)} posts collected. Adjust scrolling logic if needed.")

    # ✅ 初始化数据存储
    post_number = [i for i in range(len(post_links))]
    content_list = []
    reactions_list = []
    comments_list = []
    media_type_list = []
    image_url_list = []
    datetime_list = []
    title_list = []
    relative_time_list = []

    # ✅ 遍历每个帖子，获取详细信息
    for link in post_links:
        print(f"📌 Processing post: {link}")
        browser.get(link)
        time.sleep(5)

        # ✅ 重新获取 page_source，避免缓存问题
        response = TextResponse(
            url=browser.current_url,
            body=browser.page_source.encode('utf-8'),
            encoding='utf-8'
        )
        selector = Selector(response)

        # ✅ 提取帖子内容
        h1_text_parts = selector.xpath('//h1[contains(@class, "_ap3a")]/text()').getall()
        h1_text = ' '.join([text.strip() for text in h1_text_parts if text.strip()])
        content_list.append(h1_text)

        # ✅ 提取点赞数
        span_text = selector.xpath('//span[contains(@class, "xdj266r")]/text()').get()
        reactions_list.append(span_text)

        # ✅ 提取时间
        time_element = selector.xpath('//time[contains(@class, "x1p4m5qa")]')
        datetime_value = time_element.xpath('./@datetime').get()
        title_value = time_element.xpath('./@title').get()
        relative_time = time_element.xpath('./text()').get()
        datetime_list.append(datetime_value)
        title_list.append(title_value)
        relative_time_list.append(relative_time)

        # ✅ 处理视频帖子
        if "/reel/" in link:
            media_type_list.append("video")
            image_url_list.append([])  # ✅ 保持一致性
            continue  # ✅ 直接跳过视频帖子，不采集图片

        # ✅ 处理图片帖子
        media_type_list.append("image")
        filtered_image_srcs = set()  # ✅ **改成 `set()` 确保不会存重复图片**
        previous_image_count = 0  # **用来检测 `?img_index` 是否重复**

        # ✅ 检测是否是多图帖子
        if browser.current_url.endswith("?img_index=1"):
            print(f"🖼️ 多图帖子检测: {link}")

            for img_index in range(1, 21, 2):  # 最多检查 10 张图片
                img_url = f"{link}?img_index={img_index}"
                browser.get(img_url)
                time.sleep(3)  # **稍作等待，防止加载不完整**

                # ✅ 再次获取最新的 HTML（避免残留前一个帖子的图片）
                response = TextResponse(
                    url=browser.current_url,
                    body=browser.page_source.encode('utf-8'),
                    encoding='utf-8'
                )
                selector = Selector(response)

                # ✅ 只获取 **当前帖子的** 图片
                image_srcs = selector.xpath('//article//img[@src]/@src').getall()

                # ✅ 过滤掉缩略图，确保只保留高清图片
                valid_images = {
                    src for src in image_srcs
                    if 'data' not in src and '2885-19' not in src and 'e15_tt6' not in src
                }

                # ✅ 如果 `valid_images` 和上一次的 `filtered_image_srcs` 一样，说明重复，提前终止
                if len(valid_images) == previous_image_count:
                    print(f"🚫 Instagram 可能在重复加载相同的图片，提前停止。")
                    break

                previous_image_count = len(valid_images)  # **更新计数**
                filtered_image_srcs.update(valid_images)  # ✅ **用 `.update()` 代替 `.extend()`，去重**
                print(f"✅ 发现 {len(valid_images)} 张图片（索引 {img_index}）")

                # **提前终止条件**：如果 Instagram 强制返回 `?img_index=1`，说明没有更多图片了
                if browser.current_url.endswith("?img_index=1") and img_index != 1:
                    print("🔄 Instagram 可能已加载完所有图片，提前停止。")
                    break

        else:
            # ✅ 单图帖子
            image_srcs = selector.xpath('//article//img[@src]/@src').getall()
            valid_images = {
                src for src in image_srcs
                if 'data' not in src and '2885-19' not in src and 'e15_tt6' not in src
            }
            filtered_image_srcs.update(valid_images)

        # ✅ 确保每个帖子有对应的图片列表
        image_url_list.append(list(filtered_image_srcs))  # **用 `list(set(...))` 确保唯一性**

    # ✅ 确保所有列表长度一致
    print(f"✅ Final collected data lengths:")
    print(f"Length of post_number: {len(post_number)}")
    print(f"Length of content_list: {len(content_list)}")
    print(f"Length of reactions_list: {len(reactions_list)}")
    print(f"Length of image_url_list: {len(image_url_list)}")
    print(f"Length of media_type_list: {len(media_type_list)}")
    print(f"Length of datetime_list: {len(datetime_list)}")
    print(f"Length of title_list: {len(title_list)}")
    print(f"Length of relative_time_list: {len(relative_time_list)}")

    # ✅ 下载主图片
    print("开始下载帖子图片")

    image_download_bar = tqdm(total=len(image_url_list), desc="下载主图片")

    for idx_post, image_url_post in enumerate(image_url_list):
        if not image_url_post:  # 防止空列表错误
            image_download_bar.update(1)
            continue

        folder_name = f"data/{idx_post}"
        os.makedirs(folder_name, exist_ok=True)

        for idx_image, url in enumerate(image_url_post):  # **不再需要去重，因为 `set()` 已去重**
            image_extension = '.png'
            image_filename = f"{folder_name}/image_{idx_image}{image_extension}"
            success = download_image(url, image_filename)

            if not success:
                image_filename = "N/A"

        image_download_bar.update(1)

    image_download_bar.close()

    # ✅ 保存到 CSV
    df = pd.DataFrame({
        'Row Number': post_number,
        'Content': content_list,
        'Likes': reactions_list,
        'Images': image_url_list,
        'Post Type': media_type_list,
        'Datetime': datetime_list,
        'Date': title_list,
        'Relative Time': relative_time_list
    })
    df.to_csv('scraped_data.csv', encoding='utf-8-sig', index=False)
    print("📂 Data saved to 'scraped_data.csv'")

    return df

In [ ]:
# 进入facebook网页，登录账号
# 配置Chrome浏览器
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode (optional)
chrome_options.add_argument('--remote-debugging-port=9222')
chrome_options.add_argument("--incognito")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=chrome_options)

browser.get("https://www.instagram.com/")
time.sleep(3)  # Give it time to load
load_cookies(browser, "cookie_ins.txt")


# service = Service('/Users/qianyuzhang/Downloads/chromedriver-mac-x64/chromedriver')
# chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
# browser = webdriver.Chrome(service=service)



# browser.get('https://www.instagram.com/nyushanghai/')

# wait = WebDriverWait(browser, 60)  # 设置最长等待时间为60秒

✅ 所有 Cookies 已成功加载！


In [ ]:
scrape_instagram()

🆕 新帖子收集: https://www.instagram.com/aloyoga/reel/DFx_IheSqLK/
🆕 新帖子收集: https://www.instagram.com/aloyoga/reel/DFwbO6gxuzN/
🆕 新帖子收集: https://www.instagram.com/alomoves/reel/DFqFNyjsQxv/
🆕 新帖子收集: https://www.instagram.com/aloyoga/p/DFn0DTMyefX/
🆕 新帖子收集: https://www.instagram.com/aloyoga/reel/DFjmLlvRHzb/
🆕 新帖子收集: https://www.instagram.com/aloyoga/p/DFdC1HAtjEr/
🆕 新帖子收集: https://www.instagram.com/aloyoga/reel/DFa_r8bzEbX/
🆕 新帖子收集: https://www.instagram.com/aloyoga/p/DFV-_O-zhfk/
🆕 新帖子收集: https://www.instagram.com/aloyoga/p/DFTUivczffD/
🆕 新帖子收集: https://www.instagram.com/aloyoga/p/DFRDtqISXE3/
🆕 新帖子收集: https://www.instagram.com/aloyoga/p/DFOMoTmJihl/
🆕 新帖子收集: https://www.instagram.com/aloyoga/p/DFL44FKSXoY/
✅ 本次滚动新增 12 个帖子，总计: 12
📋 当前收集的所有帖子: ['https://www.instagram.com/aloyoga/reel/DFx_IheSqLK/', 'https://www.instagram.com/aloyoga/reel/DFwbO6gxuzN/', 'https://www.instagram.com/alomoves/reel/DFqFNyjsQxv/', 'https://www.instagram.com/aloyoga/p/DFn0DTMyefX/', 'https://www.instagram.com/aloyog

下载主图片:   0%|          | 0/100 [00:00<?, ?it/s]

📂 Data saved to 'scraped_data.csv'


,Row Number,Content,Likes,Images,Post Type,Datetime,Date,Relative Time
0,0,taking time for yourself isn’t selfish - it’s ...,521,[],video,2025-02-07T18:06:30.000Z,"Feb 7, 2025",13 hours ago
1,1,your yoga practice is everything 🤍,"2,186",[],video,2025-02-07T03:33:39.000Z,"Feb 7, 2025",1 day ago
2,2,for anyone who says yoga isn't a workout 💁‍♀️ ...,"1,729",[],video,2025-02-04T16:55:22.000Z,"Feb 4, 2025",3 days ago
3,3,slow mindful mornings make for more peaceful d...,"3,037",[https://scontent-iad3-2.cdninstagram.com/v/t5...,image,2025-02-03T19:16:53.000Z,"Feb 3, 2025",4 days ago
4,4,yoga through eyes ✨💫,"1,405",[],video,2025-02-02T04:00:17.000Z,"Feb 2, 2025",6 days ago
...,...,...,...,...,...,...,...,...
95,95,in stillness you receive the answers you need ...,annesivaasen,[https://scontent-iad3-2.cdninstagram.com/v/t5...,image,2024-10-25T20:00:51.000Z,"Oct 25, 2024","October 25, 2024"
96,96,our dream yoga mood board 🤎,marine_chpn,[https://scontent-iad3-1.cdninstagram.com/v/t5...,image,2024-10-24T23:15:33.000Z,"Oct 24, 2024","October 24, 2024"
97,97,show up for yourself 💪 join for a fearless flo...,invertedyogi,[https://scontent-iad3-2.cdninstagram.com/v/t5...,image,2024-10-23T20:43:09.000Z,"Oct 23, 2024","October 23, 2024"
98,98,the alo gift guide is here ✨ find the perfect ...,"2,474",[https://scontent-iad3-2.cdninstagram.com/v/t5...,image,2024-10-22T16:52:21.000Z,"Oct 22, 2024","October 22, 2024"


# download images to Google Drive

In [ ]:
!zip -r data.zip data

  adding: data/ (stored 0%)
  adding: data/95/ (stored 0%)
  adding: data/95/image_0.png (deflated 2%)
  adding: data/95/image_1.png (deflated 1%)
  adding: data/8/ (stored 0%)
  adding: data/8/image_0.png (deflated 0%)
  adding: data/38/ (stored 0%)
  adding: data/38/image_0.png (deflated 1%)
  adding: data/38/image_1.png (deflated 1%)
  adding: data/65/ (stored 0%)
  adding: data/65/image_0.png (deflated 0%)
  adding: data/88/ (stored 0%)
  adding: data/88/image_0.png (deflated 0%)
  adding: data/54/ (stored 0%)
  adding: data/54/image_0.png (deflated 18%)
  adding: data/54/image_1.png (deflated 15%)
  adding: data/49/ (stored 0%)
  adding: data/49/image_0.png (deflated 1%)
  adding: data/78/ (stored 0%)
  adding: data/78/image_0.png (deflated 1%)
  adding: data/78/image_1.png (deflated 0%)
  adding: data/70/ (stored 0%)
  adding: data/70/image_0.png (deflated 0%)
  adding: data/70/image_1.png (deflated 0%)
  adding: data/36/ (stored 0%)
  adding: data/36/image_0.png (deflated 1%)
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp data.zip /content/drive/MyDrive/